In [3]:
import torch
import sys
import re
import torch.nn as nn
from torch.nn import DataParallel
# from torch.nn import TransformerDecoder, TransformerDecoderLayer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cpu':
    file_path = './kaggleData/data/eng-cmn.txt'
else:
    file_path = '/kaggle/input/eng-cmn/eng-cmn.txt'
    sys.path.append('/kaggle/input/eng-cmn')


text_pairs = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:

        parts = line.strip().split('\t')
        

        if len(parts) < 3:
            continue
            
        en_text = parts[0].strip()
        zh_text = parts[1].strip()
        

        if en_text and zh_text:

            text_pairs.append((zh_text, en_text))

def split_chinese_english(s):

    pattern = re.compile(r'([\u4e00-\u9fff]+)|([^\u4e00-\u9fff]+)')
    result = []
    for block in pattern.findall(s):
        chinese_part, non_chinese_part = block
        if chinese_part:
            
            result.extend(list(chinese_part))
        elif non_chinese_part:

            subparts = non_chinese_part.lower().split()
            for sub in subparts:

                processed = re.sub(r'[^a-zA-Z]', '', sub)
                if processed:
                    result.append(processed)
    return result
    


In [4]:
from collections import defaultdict

def build_vocab(text_pairs, unknown = True):
    '''
    text_pairs: raw cn-eng text pair
    unknown: whether to deal with unkonwn words
    '''
    vocab = defaultdict(lambda: len(vocab))
    special_tokens = ["<pad>", "<sos>", "<eos>", "<sep>"]
    for token in special_tokens:
        vocab[token]

    for ch, en in text_pairs:
        ch = split_chinese_english(ch)
        for char in ch:
            vocab[char]
        en = split_chinese_english(en)
        for char in en:
            vocab[char]
    # add <unk> to the las of the vocab to deal with unknown words
    if unknown:    
        vocab['<unk>']
    
    # freeze vocab (check usage of 'defaultdict')
    vocab.default_factory = None
    
    return vocab

def text_to_ids(text, vocab):
    unk_id = vocab['<unk>']
    split = split_chinese_english(text)
    # print(split)
    ids = [vocab.get(id, unk_id) for id in split]
    return ids

In [5]:
# build vocab
vocab = build_vocab(text_pairs)
len(vocab)

10738

In [6]:
# build dataset

from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, pairs, vocab, max_len=50):
        self.pairs = pairs
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        ch, en = self.pairs[idx]

        input_ids_cn_eng = (
            [self.vocab["<sos>"]] +
            text_to_ids(ch, self.vocab) +
            [self.vocab["<sep>"]] +
            text_to_ids(en, self.vocab) +
            [self.vocab["<eos>"]]
        )

        input_ids_cn_eng = input_ids_cn_eng[:self.max_len]
        src = input_ids_cn_eng[:-1]
        tgt = input_ids_cn_eng[1:]

        # pad to max_len
        # ================================================================
        src += [self.vocab["<pad>"]]* (self.max_len - len(src))
        tgt += [self.vocab["<pad>"]]* (self.max_len - len(tgt))
        # ================================================================

        return torch.tensor(src), torch.tensor(tgt)

In [14]:
# hyper params

vocab_size = len(vocab)
eval_num = 100
BATCH_SIZE = 32
MAX_LEN = 49
model_max_seq=128
max_epoch = 4
num_heads = 2

dataset = TranslationDataset(text_pairs, vocab, MAX_LEN)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [15]:
# define a real gpt:
from BuildingBlocks import MultiHeadAttention

# gpt config:
GPT_CONFIG_124M = {
"vocab_size": len(vocab), # Vocabulary size
"context_length": 128, # Context length: max sequence length the model can handle 
"emb_dim": 64, # Embedding dimension
"n_heads": 4, # Number of attention heads
"n_layers": 4, # Number of layers
"drop_rate": 0.1, # Dropout rate
"qkv_bias": False # Query-Key-Value bias
}

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True)
        # unbiased=False: divided by n instead of n-1
        var = x.var(dim = -1, keepdim = True, unbiased = False)
        norm_x = (x-mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift
    
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5*x*(1+torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 *torch.pow(x, 3))
        ))
    
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            # (2, 3, 768)->(2, 3, 3072)
            nn.Linear(cfg["emb_dim"], 4*cfg["emb_dim"]),
            # (2,3,3072) -> (2,3,3072)
            GELU(),
            # (2,3,3072)->(2,3,768)
            nn.Linear(4*cfg["emb_dim"], cfg["emb_dim"]),
        )
    def forward(self, x):
        return self.layers(x)
    
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads = cfg["n_heads"],
            dropout = cfg["drop_rate"],
            qkv_bias = cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
    def forward(self, x):

        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x
    
# GPT DEF
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias = False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device = in_idx.device)
        )

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [18]:
import time

1744950788.8046644

In [35]:

# initialize model and start training

# model = SimpleGPTTranslator(vocab_size=vocab_size).to(device)
model = GPTModel(GPT_CONFIG_124M).to(device)

K = torch.cuda.device_count()

if K > 1:
    print(f'======= training in {K} GPUs =======')
    model = DataParallel(model, device_ids = list(range(K)))
    

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# criterion = nn.CrossEntropyLoss(ignore_index=vocab["<pad>"])
criterion = nn.CrossEntropyLoss(ignore_index = -100)
ignore_index = [0] # ignore <pad>


# training loop cn-eng:
for epoch in range(max_epoch):
    total_loss = 0
    start = time.time()
    for src, tgt in dataloader:
        
        src = src.to(device)
        tgt = tgt.to(device)
        # print(tgt)
        msk = torch.isin(tgt, torch.tensor(ignore_index).to(device))
        tgt[msk] = -100
        # print(tgt)
        # print(src.shape)
        optimizer.zero_grad()
        # print(src)
        
        output = model(src)
        # output = model_dp(src)
        
        # print(output.shape)
        loss = criterion(output.view(-1, vocab_size), tgt.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    end = time.time()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}, Time: {end-start:.4f}s")

# # training loop reverse: eng-cn
# for epoch in range(max_epoch):
#     total_loss = 0
#     for src, tgt in dataloaderreverse:
#         src = src.to(device)
#         tgt = tgt.to(device)
#         # print(tgt)
#         msk = torch.isin(tgt, torch.tensor(ignore_index).to(device))
#         tgt[msk] = -100
#         # print(tgt)
#         # print(src.shape)
#         optimizer.zero_grad()
#         # print(src)
#         output = model(src)
#         # print(output.shape)
#         loss = criterion(output.view(-1, vocab_size), tgt.view(-1))
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

# save model


Epoch 1, Loss: 5.3599, Time: 15.8013s
Epoch 2, Loss: 4.3952, Time: 15.7679s


KeyboardInterrupt: 

In [24]:
# save model
torch.save(model.module.state_dict(), "gpt2_rev_weights.pth")

In [33]:
# reload model
# model = SimpleGPTTranslator(vocab_size=vocab_size).to(device)
model = GPTModel(GPT_CONFIG_124M).to(device)
model = DataParallel(model, device_ids = list(range(K)))
model.module.load_state_dict(torch.load('gpt2_rev_weights.pth', map_location=device, weights_only=True))
# model.load_state_dict(torch.load('gpt2_weights.pth', map_location=device))

<All keys matched successfully>

In [27]:
# translate chinese to english
def translate_cn_eng(model, ch_text, vocab, max_len=80):
    model.to(device)
    model.eval()

    input_ids = (
        [vocab["<sos>"]] +
        text_to_ids(ch_text, vocab) +
        [vocab["<sep>"]]
    )
    input_tensor = torch.tensor([input_ids], device=device)


    for _ in range(max_len):
        if len(input_ids) >50:
            # print('max length hit')
            break
        output = model(input_tensor)

        # extract the last words of output as the predicted words
        # [0, -1]: 0 for batch, since we only have 1 data per batch
        next_token = output[0, -1].argmax().item()
        # print(next_token)
        input_ids.append(next_token)
        if next_token == vocab["<eos>"]:
            print('predict ends')
            break
        

        input_tensor = torch.tensor([input_ids], device=device)
    

    sep_pos = input_ids.index(vocab["<sep>"])
    en_ids = input_ids[sep_pos+1:-1]
    

    id_to_token = {v: k for k, v in vocab.items()}
    return " ".join([id_to_token[id] for id in en_ids])


# translate english to chinese (deprecated)
def translate_eng_cn(model, eng_text, vocab, max_len=80):
    model.to(device)
    model.eval()

    input_ids = (
        [vocab["<sos>"]] +
        text_to_ids(eng_text, vocab) +
        [vocab["<sep>"]]
    )
    input_tensor = torch.tensor([input_ids], device=device)
    # print(input_tensor.device)
    # input_tensor = input_tensor.to(device)
    

    for _ in range(max_len):
        if len(input_ids) >50:
            # print('max length hit')
            break
        output = model(input_tensor)

        # extract the last words of output as the predicted words
        # [0, -1]: 0 for batch, since we only have 1 data per batch
        next_token = output[0, -1].argmax().item()
        # print(next_token)
        input_ids.append(next_token)
        if next_token == vocab["<eos>"]:
            print('predict ends')
            break
        

        input_tensor = torch.tensor([input_ids], device=device)
    

    sep_pos = input_ids.index(vocab["<sep>"])
    ch_ids = input_ids[sep_pos+1:-1]
    

    id_to_token = {v: k for k, v in vocab.items()}
    return "".join([id_to_token[id] for id in ch_ids])



In [34]:
translate_cn_eng(model, '牛', vocab)

predict ends


'how much'

In [32]:
# eval
# chinese to english
import numpy as np
for i in np.random.randint(0, len(text_pairs), eval_num):
    cn = text_pairs[i][0]
    en = text_pairs[i][1]
    print(f'> {cn}')
    print(f'= {en}')
    print(f'< {translate_cn_eng(model, cn, vocab)}')
    print('='*20)


> 这是他们应得的。
= They deserve it.
predict ends
< this is the same
> 乡间有很多树。
= There are lots of trees in the countryside.
predict ends
< there are many books
> 據說她愛他。
= It's said that she loves him.
predict ends
< he knows him
> 你去哪儿？
= Where are you going?
predict ends
< where are you going
> 火車上有預訂的座位嗎？
= Are there reserved seats on the train?
predict ends
< is the desk in the world
> 他們那麼年輕。
= They were so young.
predict ends
< they are they
> 我们不能仅仅就这样炒了汤姆
= We can't just fire Tom.
predict ends
< we cant tell tom
> 我不是在罵你。
= I was not scolding you.
predict ends
< i dont have you
> 这汤里盐放多了。
= There's too much salt in this soup.
predict ends
< how much did tom
> 您会使用计算机吗？
= Can you use a computer?
predict ends
< do you want to buy a little more coffee
> 湯姆讓瑪麗回家。
= Tom let Mary go home.
predict ends
< tom went to go home
> 我一到伦敦就会给你留言的。
= As soon as I get to London, I'll drop you a line.
predict ends
< i wish you can help the same age
> 我把我借来的刀还了。
= I returned the knife which I had borrow

In [ ]:
# # english to chinese
# import numpy as np
# for i in np.random.randint(0, len(text_pairs), eval_num):
#     cn = text_pairs[i][0]
#     en = text_pairs[i][1]
#     print(f'> {en}')
#     print(f'= {cn}')
#     print(f'< {translate_eng_cn(model, en, vocab)}')
#     print('='*20)